In [1]:
import sys
sys.path.append("..")

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from src.dataset import HierarchicalDataset
import pandas as pd

In [4]:
dataset = HierarchicalDataset(config_dir="../../config/catalog.yml", 
                              cases_dir="../../data/COVID-19-up-to-date.csv", 
                              ifr_dir="../../data/weighted_fatality.csv", 
                              serial_interval_dir="../../data/serial_interval.csv", 
                              interventions_dir="../../data/interventions.csv", num_covariates=7)

../src/dataset.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ifr["country"][ifr["country"] == "United Kingdom"] = "United_Kingdom"
../src/dataset.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covariates[covariate_name][ind] = covariates["lockdown"][ind]


In [41]:
stan_data = dataset.get_stan_data(75)

../Python/dataset.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases["date"] = cases["dateRep"].apply(pd.to_datetime, format="%d/%m/%Y")
../Python/dataset.py:138: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cases["t"] = cases["date"].apply(lambda v: dt_to_dec(v))


First non-zero cases is on day 58, and 30 days before 5 days is day 52
Denmark has 37 of data
First non-zero cases is on day 31, and 30 days before 5 days is day 27
Italy has 62 of data
First non-zero cases is on day 28, and 30 days before 5 days is day 46
Germany has 43 of data
First non-zero cases is on day 32, and 30 days before 5 days is day 40
Spain has 49 of data
First non-zero cases is on day 31, and 30 days before 5 days is day 43
United_Kingdom has 46 of data
First non-zero cases is on day 25, and 30 days before 5 days is day 38
France has 51 of data
First non-zero cases is on day 58, and 30 days before 5 days is day 55
Norway has 34 of data
First non-zero cases is on day 35, and 30 days before 5 days is day 49
Belgium has 40 of data
First non-zero cases is on day 57, and 30 days before 5 days is day 53
Austria has 36 of data
First non-zero cases is on day 32, and 30 days before 5 days is day 49
Sweden has 40 of data
First non-zero cases is on day 57, and 30 days before 5 days

In [42]:
stan_data

{'M': 11,
 'p': 7,
 'x1': array([-1.97350876e-01, -1.92017069e-01, -1.86683261e-01, -1.81349454e-01,
        -1.76015647e-01, -1.70681839e-01, -1.65348032e-01, -1.60014224e-01,
        -1.54680417e-01, -1.49346609e-01, -1.44012802e-01, -1.38678994e-01,
        -1.33345187e-01, -1.28011379e-01, -1.22677572e-01, -1.17343764e-01,
        -1.12009957e-01, -1.06676149e-01, -1.01342342e-01, -9.60085345e-02,
        -9.06747270e-02, -8.53409195e-02, -8.00071121e-02, -7.46733046e-02,
        -6.93394971e-02, -6.40056896e-02, -5.86718822e-02, -5.33380747e-02,
        -4.80042672e-02, -4.26704598e-02, -3.73366523e-02, -3.20028448e-02,
        -2.66690374e-02, -2.13352299e-02, -1.60014224e-02, -1.06676149e-02,
        -5.33380747e-03, -8.84945033e-18,  5.33380747e-03,  1.06676149e-02,
         1.60014224e-02,  2.13352299e-02,  2.66690374e-02,  3.20028448e-02,
         3.73366523e-02,  4.26704598e-02,  4.80042672e-02,  5.33380747e-02,
         5.86718822e-02,  6.40056896e-02,  6.93394971e-02,  7.4

In [43]:
# remove travel bans and sport
stan_data["covariate2"] = 0*stan_data["covariate2"]
stan_data["covariate4"] = 0*stan_data["covariate4"]

# self-isolating if ill
stan_data["covariate2"] = stan_data["covariate7"]

In [44]:
#create the any intervention covariate
stan_data["covariate4"] = 1*(stan_data["covariate1"]+
                            stan_data["covariate3"]+
                            stan_data["covariate4"]+
                            stan_data["covariate5"]+
                            stan_data["covariate6"] > 1)

In [45]:
stan_data["covariate7"]=0*stan_data["covariate7"]

In [46]:
import pystan


In [28]:
sm = pystan.StanModel(file="../stan-models/base.stan")

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_cac77d4c122b5d474c43fdbfc9c71646 NOW.


In [48]:
fit = sm.sampling(data=stan_data,iter=40,warmup=20,chains=2)
print(fit)

To run all diagnostics call pystan.check_hmc_diagnostics(fit)


Inference for Stan model: anon_model_cac77d4c122b5d474c43fdbfc9c71646.
2 chains, each with iter=40; warmup=20; thin=1; 
post-warmup draws per chain=20, total post-warmup draws=40.

                      mean se_mean      sd    2.5%     25%     50%       75%     97.5%  n_eff   Rhat
mu[1]                 3.79    0.34     1.7    2.38    2.97    3.43      3.93     10.58     25   1.01
mu[2]                 3.77    0.07    0.42    3.25    3.51    3.66       4.0      5.11     31   0.99
mu[3]                 4.74     0.3    1.56    2.86    3.92    4.57      4.92     10.78     27   1.02
mu[4]                 4.93    0.26    1.32    3.61    4.29    4.66      5.25     10.22     26   1.03
mu[5]                 3.72    0.12    0.69    2.96    3.42    3.58      3.81      6.29     31   0.99
mu[6]                 3.96    0.07     0.4     3.3    3.65    3.97      4.18      4.98     36   1.02
mu[7]                 3.23    0.29    1.46    1.63    2.23    2.81      3.67      8.01     25   0.99
mu[8]      

In [51]:
# save it to the file for later use
import pickle
with open('../results/example.pkl', 'wb') as f:
    pickle.dump(sm, f)

In [52]:
# import arviz as az
fit.extract()

OrderedDict([('mu',
              array([[ 3.41917996,  3.48697729,  4.76090161,  4.8512507 ,  3.72424283,
                       3.9494985 ,  1.75953056,  3.6855206 ,  3.92049174,  4.79535165,
                       3.0182314 ],
                     [ 4.90598031,  3.99758166,  4.81717883,  4.90183869,  4.09055337,
                       4.01502429,  3.86591745,  5.43969272,  4.15387292,  3.30415317,
                       4.34042948],
                     [ 3.80693479,  3.72003751,  4.69204753,  4.70395464,  3.47992437,
                       4.02888883,  1.9339287 ,  3.78757421,  4.57840194,  4.72848874,
                       3.5484058 ],
                     [ 3.9220363 ,  3.80259963,  4.92890336,  4.81474014,  3.17186569,
                       3.88516326,  3.64277817,  3.81545212,  2.75309577,  3.61036552,
                       3.34672128],
                     [ 4.29088401,  3.58948378,  3.61859651,  4.54447379,  3.88433848,
                       3.37130216,  2.3790803 ,  3.77